# Anime Project

### Tasks:
1. Extract data from ani-watch anime
2. Transform the useful data
3. Load it into a database

## Staging

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
URL = 'https://aniwatch.to/recently-updated'
res = requests.get(URL)

In [3]:
soup = BeautifulSoup(res.text, "html.parser")

## Extract

In [4]:
recent_animes = soup.body.find(class_="film_list-wrap").find_all(class_="flw-item")

In [5]:
recent_animes[1]

<div class="flw-item flw-item-big">
<div class="film-poster">
<div class="tick ltr">
<div class="tick-item tick-sub"><i class="fas fa-closed-captioning mr-1"></i>19</div>
<div class="tick-item tick-eps">54</div>
</div>
<img alt="Soul Land 2: Peerless Tang Sect" class="film-poster-img lazyload" data-src="https://img.flawlessfiles.com/_r/300x400/100/78/67/78670f95e8a9cdaaedc82e50c998674d/78670f95e8a9cdaaedc82e50c998674d.png"/>
<a class="film-poster-ahref item-qtip" data-id="18416" href="/soul-land-2-peerless-tang-sect-18416" title="Soul Land 2: Peerless Tang Sect"><i class="fas fa-play"></i></a>
</div>
<div class="film-detail">
<h3 class="film-name"><a class="dynamic-name" data-jname="Douluo Dalu II: Jueshi Tangmen" href="/soul-land-2-peerless-tang-sect-18416" title="Soul Land 2: Peerless Tang Sect">Soul Land 2: Peerless Tang Sect</a></h3>
<div class="description">
            A strange continent in another world.  No magic, no battle qi, no martial arts, only martial spirits. Ten thousa

In [6]:
data = []

for anime in recent_animes:
    try:
        id =int(anime.find('a', class_='film-poster-ahref')['data-id'])
        name = anime.find('a', class_='dynamic-name').text
        sub = anime.find('div',class_='tick-sub')
        sub = int(sub.text) if sub else sub
        dub = anime.find('div',class_='tick-dub')
        dub = int(dub.text) if dub else dub
        description = anime.find('div', class_='description')
        description = description.text.strip() if description else None
        img_src = anime.find('img')
        img_src = img_src['data-src'] if img_src else None

        data.append((id,name,sub,dub,description,img_src))
    except:
        with open("log.txt",'a') as file:
            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            file.write(f'[{now}] ERROR: unable to extract data from following anime:\n')
            file.write(f"{str(id)}, {name}\n")
            file.write(f"{str(id)}:\n {anime}\n\n")

In [7]:
import pymysql

cnx = pymysql.connect(host='myhpnet.duckdns.org',
                      user='surya',
                      password='Asspr2320',
                      port=3306,
                      database='myhpnet')
with cnx:
    with cnx.cursor() as cursor:
        query="REPLACE INTO anime_watch(id, name, sub, dub, description, images)\
            VALUES (%s,%s,%s,%s,%s,%s)"
        for d in data:
            cursor.execute(query,d)
        cnx.commit()

## Transform

In [13]:
watching_anime = {"One Peice", "The Demon Sword Master of Excalibur Academy", \
    "Berserk of Gluttony", "Am I Actually the Strongest?",\
        "Bleach: Thousand-Year Blood War - The Separation"}

In [14]:
anime_names = list(anime['name'])

In [20]:
# Listing anime with new episodes to watch
prev_anime = "Berserk of Gluttony"
new_animes = anime_names[:anime_names.index(prev_anime)]

anime_to_watch = []

for anime in new_animes:
    if anime in watching_anime:
        anime_to_watch.append(anime)

anime_to_watch

['The Demon Sword Master of Excalibur Academy']

In [36]:
animes['datetime'].diff().dt